# Chapter 01: Differentiation Matgrices



In [ ]:
using PyPlot
using SparseArrays
using LinearAlgebra

## Program 1

In [ ]:
# Convergence of fourth-order finite differences

function diff4th(N, f, df, a=-π, b=π)
    h =(b-a) / N
    x = a .+ (1:N) * h
    u = f.(x)
    du = df.(x)
    e = ones(N)
    D = sparse(1:N, [2:N; 1], 2*e/3, N, N) - 
        sparse(1:N, [3:N; 1; 2], e/12, N, N) 
    D = (D-D') / h
    return norm(D*u - du)
    
end


In [ ]:
f(x) = exp(sin(x))
df(x) = cos(x)*f(x)
N = 2 .^(3:12)
ε = diff4th.(N, f, df);

loglog(N, ε, "k.")
grid(linestyle=":")
plot(N, N.^(-4), "--")
text(105, 5e-8, "\$N^{-4}\$")

## Program 2

In [ ]:
using ToeplitzMatrices

In [ ]:
function periodicSpectral(N, f, df; plt=false)
    h = 2π/N
    x = -π .+ (1:N)*h
    u = f.(x)
    du = df.(x)
    column = [0; .5*(-1).^(1:N-1).*cot.((1:N-1)*h/2)]
    D = Toeplitz(column, column[[1; N:-1:2]])
    
    du1 = D*u
    ε = du1 - du
    if plt
        xx = range(-2π, 2π, length=N*30) 
        yy = df.(xx)
        plot(xx, yy)
        plot(x, du1, "ro")
        imax = argmax(abs.(ε))
        axvline(x[imax])
    end
    return norm(ε)
end

In [ ]:
periodicSpectral(26, f, df, plt=true)

In [ ]:
N = 2:2:100
ε = periodicSpectral.(N, f, df);

loglog(N, ε, "k.")
grid(linestyle=":")
#plot(N, N.^(-4), "--")
#text(105, 5e-8, "\$N^{-4}\$")

# Exercícios

## Exercício 1.1


In [ ]:
using Polynomials

In [ ]:
# Calculates the numerator of the kth lagrange polynomial
function lagrnum(pts, k)
    npts = length(pts)
    roots = zeros(Int, npts-1)
    idx = 1
    for i in 1:k-1
        roots[idx] = pts[i]
        idx += 1
    end
    for i in k+1:npts
        roots[idx] = pts[i]
        idx += 1
    end
    return fromroots(roots)
end
# Calculates the denominator of the kth lagrange polynomial
function lagrden(pts, k)
    npts = length(pts)
    den = 1
    for i in 1:k-1
        den *= (pts[k]-pts[i])
    end
    for i in k+1:npts
        den *= (pts[k]-pts[i])
    end
    return den
end

    


In [ ]:
lagrden([1,2,3], 2)

In [ ]:
function dcoeffs(n)
    pts = -n:n
    npts = length(pts)
    
    lnum = [lagrnum(pts, k) for k in 1:npts]
    lden = [lagrden(pts, k) for k in 1:npts]
    dnum = [derivative(l) for l in lnum]
    coefs = [derivative(lnum[k])(0)/lden[k] for k in 1:npts]
    return coefs
end

function diffmat(n, N, h=1.0)
    
    p = dcoeffs(n)/h
    npts = 2n + 1
    vc = zeros(N)
    vr = zeros(N)
    
    for i in 1:n
        vr[i+1] = p[i+n+1]
        vc[i+1] = p[n+1-i]
        vc[N-i+1] = p[i+n+1]
        vr[N-i+1] = p[n+1-i]
    end
    return Toeplitz(vc, vr)
end
    
    

In [ ]:
diffmat(1, 8, 1.0)

In [ ]:
function test_diff(n, N, f, df, a=-π, b=π; plt=false)
    h =(b-a) / N
    x = a .+ (1:N) * h
    u = f.(x)
    du = df.(x)
    D = diffmat(n, N, h)
    du1 = D*u
    if plt
        xx = range(a, b, length=N*20)
        dyy = df.(xx)
        plot(xx, dyy)
        plot(x, du1, "ro")
    end
    return norm(du1 - du)
end

In [ ]:
N = 2 .^(3:12)
ε₁ = test_diff.(1, N, f, df);
ε₂ = test_diff.(2, N, f, df);
ε₃ = test_diff.(3, N, f, df);
ε₄ = test_diff.(4, N, f, df);
loglog(N, ε₁, "o:", label="n=1")
loglog(N, ε₂, "x:", label="n=2")
loglog(N, ε₃, "+:", label="n=3")
loglog(N, ε₄, "s:", label="n=4")
grid(linestyle=":")
legend()
#plot(N, N.^(-4), "--")
#text(105, 5e-8, "\$N^{-4}\$")

## Exercício 1.2

Por construção, a matriz é de Toeplitz e na sua construção, a partir da primeira coluna e primeira linha vê-se que a matriz é circulante.
Na função `diffmat` existe o seguinte trecho:

```julia
    for i in 1:n
        vr[i+1] = p[i+n+1]
        vc[i+1] = p[n+1-i]
        vc[N-i+1] = p[i+n+1]
        vr[N-i+1] = p[n+1-i]
    end
```

## Exercício 1.3

In [ ]:
periodicSpectral.(14, f, df, plt=true)

A função 

$$
e^{\sin x}
$$

tem o jeitão da função seno. Ao se introduzir um cosseno, a aproximação não melhora significativamente. Ao se introduzir um novo seno na séria a coisa melhora.


## Exercício 1.4

In [ ]:
N = 2 .^(3:16)
ε = diff4th.(N, f, df);

loglog(N, ε, "k.")
grid(linestyle=":")
plot(N, N.^(-4), "--")
text(105, 5e-8, "\$N^{-4}\$")


**Floating point errors**

In [ ]:
g1(x) = exp(sin(x)^2)
dg1(x) = g1(x) * 2*sin(x)*cos(x)
N = 2 .^(3:12)
ε = diff4th.(N, g1, dg1);

loglog(N, ε, "k.")
grid(linestyle=":")
plot(N, N.^(-4), "--")
text(105, 5e-8, "\$N^{-4}\$")

In [ ]:
g2(x) = exp(sin(x)*abs(sin(x)))
dg2(x) = g2(x) * (cos(x)*abs(sin(x)) + sin(x)*sign(x)*cos(x))
N = 2 .^(3:12)
ε = diff4th.(N, g2, dg2);

loglog(N, ε, "k.")
grid(linestyle=":")
plot(N, N.^(-4), "--")
text(105, 5e-8, "\$N^{-4}\$")
plot(N, N.^(-1), "--")
text(105, 1e-3, "\$N^{-1}\$")


In [ ]:
x = -π:0.001:π
y = g2.(x)
plot(x,y)